In [1]:
import psycopg2
import pandas as pd
import numpy as np
from psycopg2 import OperationalError
import datetime

In [ ]:
# Параметры подключения
host = "rc1a-p8bp15mmxsfwpbt0.mdb.yandexcloud.net"
port = 6432
database = "db1"
user = "test_user"
password = "j2M{CnnFq@"

try:
    # Устанавливаем соединение
    connection = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    print("Соединение успешно установлено!")

    # Выполняем запрос и сохраняем результат в DataFrame
    query = "SELECT * FROM test.managers;"
    df_managers = pd.read_sql(query, connection)

    query = "SELECT * FROM test.chat_messages;"
    df_messages = pd.read_sql(query, connection)

    query = "SELECT * FROM test.rops;"
    df_rops = pd.read_sql(query, connection)
        
except OperationalError as e:
    print(f"Ошибка подключения: {e}")
finally:
    if connection:
        connection.close();

In [5]:
#    Подготавливаем исходный DataFrame:
#    - берем нужные столбцы ('entity_id', 'created_by', 'created_at') из df_messages
#    - переименовываем в ['deal', 'sender', 'sent_at']
#    - переводим created_at из секунд (unix time) в datetime
#    - сортируем по deal и времени отправки (sent_at)
#    - меняем тип данных в df_managers['rop_id'] со str на int для будущего джойна

df = df_messages[['entity_id', 'created_by', 'created_at']].copy()
df['created_at'] = pd.to_datetime(df['created_at'], unit='s')
df.columns = ['deal', 'sender', 'sent_at']
df.sort_values(by=['deal', 'sent_at'], inplace=True)

df_managers['rop_id'] = df_managers['rop_id'].astype('int64')

In [7]:
#    Определяем, КТО (sender) отправил предыдущее сообщение
#    и КОГДА оно было отправлено (sent_at) — для каждой строки.
#    Используем shift(1) внутри группировки по 'deal'.
#    - received_from: sender предыдущего сообщения в этом же диалоге
#    - received_at:   время предыдущего сообщения

df['received_from'] = df.groupby('deal')['sender'].shift(1)
df['received_at']   = df.groupby('deal')['sent_at'].shift(1)

df['received_from'] = df.groupby('deal')['sender'].shift(1)
df['received_at'] = df.groupby('deal')['sent_at'].shift(1)

In [9]:
#    Прибавляем 3 часа к полям sent_at и received_at.
#    Это корректировка часового пояса

df['received_at'] = df['received_at'] + pd.to_timedelta('03:00:00')
df['sent_at'] = df['sent_at'] + pd.to_timedelta('03:00:00')

In [11]:
#    Фильтрация сообщений:
#    - Берём только строки, где:
#        (sender != received_from)  => не подряд от одного отправителя
#        (sender != 0)             => отправитель - не "0" (0 = клиент)
#    - Исключаем строки, у которых received_from = NaN
#      (это бывает у первого сообщения в диалоге, где предыдущего просто нет)


df = df.loc[
    (df['sender'] != df['received_from']) & (df['sender'] != 0)
]
df = df.loc[
    ~df['received_from'].isna()
]
df

,deal,sender,sent_at,received_from,received_at
4102,35810509,10262493,2024-10-26 18:21:05,0.0,2024-10-26 18:19:26
6639,35810509,10262493,2024-10-26 18:36:46,0.0,2024-10-26 18:34:46
17852,35945535,10262493,2024-10-27 20:42:07,0.0,2024-10-27 20:38:36
14833,36099337,11241942,2024-10-27 16:25:47,0.0,2024-10-27 16:21:56
15039,36099337,11241942,2024-10-27 16:29:37,0.0,2024-10-27 16:27:14
...,...,...,...,...,...
18920,37557147,7996770,2024-10-27 23:53:50,0.0,2024-10-27 23:53:36
18917,37557147,7996770,2024-10-27 23:54:37,0.0,2024-10-27 23:54:35
18795,37557149,10465254,2024-10-27 23:33:40,0.0,2024-10-27 23:05:09
18794,37557161,10262493,2024-10-27 23:33:56,0.0,2024-10-27 23:11:16


In [13]:
#    Корректируем «ночное» время (00:00:01 - 09:29:59) к 09:30:00 того же дня.
#    Сначала определяем булевы флаги, попадает ли значение в интервал ночи:
#      sent_at_night     = True/False
#      received_at_night = True/False
#
#    Затем, где флаг True - сдвигаем время на 09:30:00.

start_time = pd.to_datetime('00:00:01').time()
end_time = pd.to_datetime('09:29:59').time()

df['sent_at_night'] = df['sent_at'].dt.time.between(start_time, end_time)
df['received_at_night'] = df['received_at'].dt.time.between(start_time, end_time)

df.loc[df['sent_at_night'] == True, 'sent_at'] = df.loc[df['sent_at_night'] == True, 'sent_at'].apply(lambda x: x.replace(hour=9, minute=30, second=0))
df.loc[df['received_at_night'] == True, 'received_at'] = df.loc[df['received_at_night'] == True, 'received_at'].apply(lambda x: x.replace(hour=9, minute=30, second=0))

df

,deal,sender,sent_at,received_from,received_at,sent_at_night,received_at_night
4102,35810509,10262493,2024-10-26 18:21:05,0.0,2024-10-26 18:19:26,False,False
6639,35810509,10262493,2024-10-26 18:36:46,0.0,2024-10-26 18:34:46,False,False
17852,35945535,10262493,2024-10-27 20:42:07,0.0,2024-10-27 20:38:36,False,False
14833,36099337,11241942,2024-10-27 16:25:47,0.0,2024-10-27 16:21:56,False,False
15039,36099337,11241942,2024-10-27 16:29:37,0.0,2024-10-27 16:27:14,False,False
...,...,...,...,...,...,...,...
18920,37557147,7996770,2024-10-27 23:53:50,0.0,2024-10-27 23:53:36,False,False
18917,37557147,7996770,2024-10-27 23:54:37,0.0,2024-10-27 23:54:35,False,False
18795,37557149,10465254,2024-10-27 23:33:40,0.0,2024-10-27 23:05:09,False,False
18794,37557161,10262493,2024-10-27 23:33:56,0.0,2024-10-27 23:11:16,False,False


In [15]:
#    Считаем разницу во времени (в минутах) между received_at (предыдущее сообщение) и sent_at (текущее).
#    Если то же число (cond_same_day), берём sent_at - received_at.
#    Если разные даты, то логика "до полуночи + после 09:30".
#
#    Формула для разных дат:
#      time_to_midnight = (23:59:59 того же дня - received_at)
#      time_from_930    = (sent_at - (sent_at дата + 09:30:00))
#    Суммируем их и переводим в минуты.

cond_same_day = df['sent_at'].dt.date.eq(df['received_at'].dt.date)

# 1) Разница, если один день
time_diff_same_day = (df['sent_at'] - df['received_at']).dt.total_seconds() / 60

# 2) Разница, если разные дни:
#    (23:59:59 того же дня - received_at) + (sent_at - 09:30 следующего дня)
time_to_midnight = (
    (df['received_at'].dt.normalize() + pd.Timedelta('23:59:59')) 
    - df['received_at']
)
time_from_930 = (
    df['sent_at'] 
    - (df['sent_at'].dt.normalize() + pd.Timedelta(hours=9, minutes=30))
)

time_diff_diff_day = (time_to_midnight + time_from_930).dt.total_seconds() / 60

# Склеиваем логику в одну серию:
df['minutes_diff'] = np.where(cond_same_day, time_diff_same_day, time_diff_diff_day)

In [17]:
#    Группируем данные по sender (менеджеру), считаем среднее (mean) по 'minutes_diff'
#    Присваиваем имя столбца 'min_avg'. Сортируем по этому среднему.
#    Затем джойним (merge) с df_managers, чтобы получить данные о менеджерах — связываем по sender = mop_id.

res = df.groupby('sender').agg(min_avg = ('minutes_diff', 'mean')).reset_index().sort_values(by='min_avg')
res = pd.merge(res, df_managers, left_on = 'sender', right_on = 'mop_id')
res = pd.merge(res, df_rops, on='rop_id')

In [19]:
res = res[['rop_name', 'name_mop', 'min_avg']]

In [21]:
res['min_avg'] = res['min_avg'].round(1)

In [23]:
res.columns = ['Руководитель', 'Манагер', 'Среднее время ответа, мин.']

In [25]:
res

,Руководитель,Манагер,"Среднее время ответа, мин."
0,Катя РОП,Настя и Даша,3.5
1,Полина РОП,Даша и Даша,4.0
2,Эля РОП,Алина и Юля,4.6
3,Эля РОП,Влада и Настя,4.7
4,Полина РОП,Мария и Соня,5.2
5,Эля РОП,Настя и Малика,5.6
6,Полина РОП,Настя и Саша,5.8
7,Полина РОП,Лиза и Ева,5.9
8,Эля РОП,Ангелина Милованова,6.1
9,Эля РОП,Даша и Карина,7.6
